In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from datetime import datetime, timedelta
import pickle
import ipywidgets as widgets
import cartopy.crs as ccrs
import cartopy.feature as cfeature

plt.rcParams['legend.fontsize'] = 10
plt.rcParams.update({'figure.autolayout': True})

## Downloading Data

In [ ]:
with open('List_of_NGL_stations.txt','r') as station_list_file:
    stations = station_list_file.read().splitlines()
print(stations, len(stations))

In [ ]:
NGL_dir = "./PreTohoku_deformation_field"
"""
-q: suppress output
-nc: avoid duplicate downloads
-P PATH: set prefix to the path where downloaded files are written.
"""
for station in stations:
    cmd = "wget -q -nc -P {0} http://geodesy.unr.edu/gps_timeseries/txyz/IGS14/{1}.txyz2".format(NGL_dir, station)
    status = os.system(cmd)
    if status == 0:
        cmd2 = "wget -q -nc -P {0} http://geodesy.unr.edu/NGLStationPages/stations/{1}.sta".format(NGL_dir, station)
        status2 = os.system(cmd2)
        if status2 != 0:
            print("Station info for {0} doesn't exist!".format(staion))
    if status != 0:
        print("Data file for {0} doesn't exist!".format(station))

In [ ]:
import glob

# absolute path to search all text files inside a specific folder
path = NGL_dir + '/' + r'*.txyz2'
datafile_names = glob.glob(path)

# print(datafile_names)

In [ ]:
stations_longenough = []
start_date = datetime(2008,1,1).date()
end_date   = datetime(2011,3,11).date()
for fname in datafile_names:
#     print(fname)
    firstline_content = []
    lastline_content = []
    
    with open(fname, 'r') as datafile:
        lastline_content = datafile.readlines()[-1].split()
        
    with open(fname, 'r') as datafile:
        firstline_content = datafile.readline().split()
    
    station = firstline_content[0]
    firstdate_string = firstline_content[1]
    lastdate_string = lastline_content[1]
    firstdate = datetime.strptime(firstdate_string, "%y%b%d").date()
    lastdate = datetime.strptime(lastdate_string, "%y%b%d").date()
    
#     print(firstdate, lastdate)
    if firstdate < start_date and lastdate >= end_date:
        stations_longenough.append(station)

print(len(stations_longenough))
print(stations_longenough)        

In [ ]:
stations_longenough_dic = {}
for station in stations_longenough:
    
    stationfile_name = NGL_dir + '/' + "{0}.sta".format(station)
    
    lat = 0.0
    lon = 0.0
    with open(stationfile_name, 'r') as stationfile:
        lines = stationfile.readlines()
        for line in lines:
            if 'Latitude' in line:
                lat = float(line.split()[1])
            if 'Longitude' in line:
                lon = float(line.split()[1])
    stations_longenough_dic.update( {station: np.array([lat, lon])} )
print(stations_longenough_dic)

In [ ]:
import cartopy.io.img_tiles as cimgt
stamen_terrain = cimgt.Stamen('terrain-background')

fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(8)

#ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax = fig.add_subplot(1, 1, 1, projection=stamen_terrain.crs)
ax.set_extent([124, 131, 32, 39], crs=ccrs.PlateCarree())

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

# Add the Stamen data at zoom level 8.
ax.add_image(stamen_terrain, 8)
ax.coastlines(resolution='10m')
gl = ax.gridlines(draw_labels=True, color='white')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlocator = mticker.FixedLocator([125, 126, 127, 128, 129, 130, 131])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylabel_style = {'size': 17, 'color': 'black'}
gl.xlabel_style = {'size': 17, 'color': 'black'}

for station in stations_longenough_dic.keys():
    print(station)
    lat, lon = stations_longenough_dic[station]
    ax.plot(lon, lat, marker='o', color='red', markersize=12,
            alpha=0.7, transform=ccrs.PlateCarree())
    ax.text(lon, lat+0.1, station,
            verticalalignment='center', horizontalalignment='right',
            transform=ccrs.PlateCarree(),
            bbox=dict(facecolor='sandybrown', alpha=0.5, boxstyle='round'))
plt.show()
fig.savefig("PreTohoku_stations_map.pdf")